# Ottenimento prossimi match
Lo script deve permettere di prendere i prossimi 10 match. 
Verrà fatta una predizione su questi match, prima deve essere calcolata la media di ciascun match che è data dai 5 match precedenti.

In [1]:
from analysis import MatchAnalysis
import pandas as pd
from my_model import MyModel
from ranking import Ranking

In [8]:
championship = pd.read_csv('files/Serie A/22_23 championship_matches.csv', index_col=0)
championship['date'] = pd.to_datetime(championship['date'], format='%Y-%m-%d')

championship.result = championship.result.astype(str)
new_matches = championship[championship.result == 'nan']

In [7]:
matches = new_matches[:10][['team1', 'team2', 'date']]

,team1,date,matchday,stadium,result,goals,team2
1,Fiorentina,2019-08-24,Giornata n. 1,Casa,P,3.0,Napoli
1,Parma,2019-08-24,Giornata n. 1,Casa,P,0.0,Juventus
1,Udinese,2019-08-25,Giornata n. 1,Casa,V,1.0,Milan
5,Torino,2019-08-25,Giornata n. 1,Casa,V,2.0,Sassuolo
1,Sampdoria,2019-08-25,Giornata n. 1,Casa,P,0.0,Lazio
...,...,...,...,...,...,...,...
39,Lecce,2020-08-02,Giornata n. 38,Casa,P,3.0,Parma
40,SPAL,2020-08-02,Giornata n. 38,Casa,P,1.0,Fiorentina
40,Genoa,2020-08-02,Giornata n. 38,Casa,V,3.0,Hellas Verona
39,Bologna,2020-08-02,Giornata n. 38,Casa,N,1.0,Torino


In [4]:
r = Ranking("Serie A", '2022-2023')
r.read_matches(seasons=4)

for i, m in matches.iterrows():
    rankH, rankA = r.get_rank(team_name1=m.team1, team_name2=m.team2)
    
    matches.at[i, 'rank_h'] = rankH
    matches.at[i, 'rank_a'] = rankA
matches

,team1,team2,date


Per ciascun match devo calcolare le statistiche date dalle medie dei 5 match precedenti e poi devo ottenere il testo da football_prediction

In [ ]:
r = Ranking("Serie A", '2022-2023')
r.read_matches(seasons=5)
ma = MatchAnalysis("files/Serie A/3 seasons.csv")
ma.set_ranking(r)
ma.read_diff_dataset()

ma.matches_fe_team['date'] = pd.to_datetime(ma.matches_fe_team['date'], format='%Y-%m-%d')
ma.create_team_dataset()

float_features_and_avg = [x for x in ma.matches_by_team[0].matches.columns if x != 'team1' and x != 'team2' and x != 'date' and x != 'result' and x != 'rank_h' and x != 'rank_a' and x != 'matchday' and x != 'stadium' and x != 'completed_passings' and x != 'total_passings' and x != 'corners']

for i, match in matches.iterrows(): 
    avgH, c = ma.get_team_by_name(match.team1).get_avg_last_X_matches(5, match.date, float_features_and_avg)
    avgA, c = ma.get_team_by_name(match.team2).get_avg_last_X_matches(5, match.date, float_features_and_avg)

    for col in float_features_and_avg: 
        diff = avgH[col] - avgA[col]
        matches.at[i, col] = diff

matches